<h1> Azure AI Agent Service </h1>

<h2>Deploying an agent with infrastructure as code</h2>

<h2>Creating a simple agent</h2>

In [ ]:
# !pip install azure-ai-projects
# !pip install azure-identity
# !python.exe -m pip install --upgrade pip
# !pip install python-dotenv 

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB 279.3 kB/s eta 0:00:07
   -- ------------------------------------- 0.1/1.8 MB 581.0 kB/s eta 0:00:04
   -------- ------------------------------- 0.4/1.8 MB 1.8 MB/s eta 0:00:01
   ----------------------- ---------------- 1.1/1.8 MB 4.3 MB/s eta 0:00:01
   ---------------------------------------  1.8/1.8 MB 6.1 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 5.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0


In [25]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential
from typing import Any
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<ProjectName>"
# HostName can be found by navigating to your discovery_url and removing the leading "https://" and trailing "/discovery"
# To find your discovery_url, run the CLI command: az ml workspace show -n {project_name} --resource-group {resource_group_name} --query discovery_url
# Project Connection example: eastus.api.azureml.ms;12345678-abcd-1234-9fc6-62780b3d3e05;my-resource-group;my-project-name
# Customer needs to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ.get("PROJECT_CONNECTION_STRING")
)

with project_client:
    # Create an instance of the CodeInterpreterTool
    code_interpreter = CodeInterpreterTool()

    # The CodeInterpreterTool needs to be included in creation of the agent
    agent = project_client.agents.create_agent(
        model="gpt-4o-mini",
        name="my-agent",
        instructions="You are helpful agent",
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )
    print(f"Created agent, agent ID: {agent.id}")

    # Create a thread
    thread = project_client.agents.create_thread()
    print(f"Created thread, thread ID: {thread.id}")

    # Create a message
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Could you please create a bar chart for the operating profit using the following data and provide the file to me? Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, Company D: $1.8 million",
    )
    print(f"Created message, message ID: {message.id}")

    # Run the agent
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    # Get messages from the thread
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

    # Get the last message from the sender
    last_msg = messages.get_last_text_message_by_role("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

    # Generate an image file for the bar chart
    for image_content in messages.image_contents:
        print(f"Image File ID: {image_content.image_file.file_id}")
        file_name = f"{image_content.image_file.file_id}_image_file.png"
        project_client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
        print(f"Saved image file to: {Path.cwd() / file_name}")

    # Print the file path(s) from the messages
    for file_path_annotation in messages.file_path_annotations:
        print(f"File Paths:")
        print(f"Type: {file_path_annotation.type}")
        print(f"Text: {file_path_annotation.text}")
        print(f"File ID: {file_path_annotation.file_path.file_id}")
        print(f"Start Index: {file_path_annotation.start_index}")
        print(f"End Index: {file_path_annotation.end_index}")
        project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=Path(file_path_annotation.text).name)

    # Delete the agent once done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

Created agent, agent ID: asst_4bUsle9VtUUKHM4R9qt8pIaz
Created thread, thread ID: thread_vGkUyo9iWrXGsxghNmiiLGGm
Created message, message ID: msg_rF3rn31nVYcGzXxpsKXK3PYN
Run finished with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_BKN7qvTWN44Ts8dEOuccCSJd', 'object': 'thread.message', 'created_at': 1741345692, 'assistant_id': 'asst_4bUsle9VtUUKHM4R9qt8pIaz', 'thread_id': 'thread_vGkUyo9iWrXGsxghNmiiLGGm', 'run_id': 'run_BhbbS4oyrULVUBFxeWboN93b', 'role': 'assistant', 'content': [{'type': 'image_file', 'image_file': {'file_id': 'assistant-3u5c14EihWjQLXQiwZTouH'}}, {'type': 'text', 'text': {'value': "I've created the bar chart for the operating profits of the companies. You can download it using the link below:\n\n[Download the operating profit bar chart](sandbox:/mnt/data/operating_profit_bar_chart.png)", 'annotations': [{'type': 'file_path', 'text': 'sandbox:/mnt/data/operating_profit_bar_chart.png', 'start_index': 156, 'end_index': 204, 'file_path

<h2>Using knowledge tools</h2>

<h3>Grounding with Bing Search</h3>

Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.</br>
At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"</br>
You need to log into your Azure subscription via the Azure CLI and set the relevant environment variables.

In [39]:
from azure.ai.projects.models import BingGroundingTool

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

In [ ]:
bing_connection = project_client.connections.get(
    connection_name=os.environ["BING_CONNECTION_NAME"]
)
conn_id = bing_connection.id

print(conn_id)

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

# Create agent with the bing tool and process assistant run
with project_client:
    agent = project_client.agents.create_agent(
        model="gpt-35-turbo",
        name="my-assistant",
        instructions="You are a helpful assistant",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )
    print(f"Created agent, ID: {agent.id}")

HttpResponseError: (model_not_found_or_ready) The model deployment provided for this resource does not exist or is not yet ready. If you created the deployment within the last 5 minutes, please wait a moment and try again.
Code: model_not_found_or_ready
Message: The model deployment provided for this resource does not exist or is not yet ready. If you created the deployment within the last 5 minutes, please wait a moment and try again.

In [ ]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="What is the top news today",
)
print(f"Created message, ID: {message.id}")

In [ ]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}")

# Retrieve run step details to get Bing Search query link
# To render the webpage, we recommend you replace the endpoint of Bing search query URLs with `www.bing.com` and your Bing search query URL would look like "https://www.bing.com/search?q={search query}"
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
print(f"Last run step detail: {run_steps_data}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Delete the assistant when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

<h3>Grounding with your files</h3>

<h4>Upload files</h4>

<h4>Azure Blob Storage</h4>

<h3>Grounding with Azure AI Search</h3>

In [41]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import AzureAISearchTool

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<ProjectName>"
# HostName can be found by navigating to your discovery_url and removing the leading "https://" and trailing "/discovery" 
# To find your discovery_url, run the CLI command: az ml workspace show -n {project_name} --resource-group {resource_group_name} --query discovery_url
# Project Connection example: eastus.api.azureml.ms;my-subscription-id;my-resource-group;my-hub-name

connection_string = os.environ["PROJECT_CONNECTION_STRING"] 

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=connection_string,
)

In [ ]:
# AI Search resource connection ID
# This code looks for the AI Search Connection ID and saves it as variable conn_id

# If you have more than one AI search connection, try to establish the value in your .env file.
# Extract the connection list.
conn_list = project_client.connections._list_connections()["value"]
conn_id = ""

# Search in the metadata field of each connection in the list for the azure_ai_search type and get the id value to establish the variable
for conn in conn_list:
    metadata = conn["properties"].get("metadata", {})
    if metadata.get("type", "").upper() == "AZURE_AI_SEARCH":
        conn_id = conn["id"]
        break

In [ ]:
# TO DO: replace this value with the connection ID of the search index
conn_id =  "/subscriptions/<your-subscription-id>/resourceGroups/<your-resource-group>/providers/Microsoft.MachineLearningServices/workspaces/<your-project-name>/connections/<your-azure-ai-search-connection-name>"

# Initialize agent AI search tool and add the search index connection ID and index name
# TO DO: replace <your-index-name> with the name of the index you want to use
ai_search = AzureAISearchTool(index_connection_id=conn_id, index_name="<your-index-name>"
query_type="<select-search-type>")

In [ ]:
agent = project_client.agents.create_agent(
    model="gpt-4o-mini",
    name="my-assistant",
    instructions="You are a helpful assistant",
    tools=ai_search.definitions,
    tool_resources = ai_search.resources,
)
print(f"Created agent, ID: {agent.id}")

In [ ]:
# Create a thread
thread = project_client.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")
 
# Create a message
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="what are my health insurance plan coverage types?",
)
print(f"Created message, message ID: {message.id}")
    
# Run the agent
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}")
 
if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

# Get messages from the thread 
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")
    
assistant_message = ""
for message in messages.data:
    if message["role"] == "assistant":
        assistant_message = message["content"][0]["text"]["value"]

# Get the last message from the sender
print(f"Assistant response: {assistant_message}")

<h2>Using action tools</h2>

<h3>Function calling</h3>

In [43]:
import json
import datetime
from typing import Any, Callable, Set, Dict, List, Optional

def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a weather API.
    # Here, we'll mock the response.
    mock_weather_data = {"New York": "Sunny, 25°C", "London": "Cloudy, 18°C", "Tokyo": "Rainy, 22°C"}
    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json

# Statically defined user functions for fast reference
user_functions: Set[Callable[..., Any]] = {
    fetch_weather,
}

In [58]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet
# from user_functions import user_functions # user functions which can be found in a user_functions.py file.

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

# Initialize agent toolset with user functions
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

agent = project_client.agents.create_agent(
    model="gpt-4o-mini", name="my-agent", instructions="You are a weather bot. Use the provided functions to help answer questions.", toolset=toolset
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_BJ2u2TabJ3sXP5SRoai7DNUD


In [ ]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Hello, send an email with the datetime and weather information in Brussels?",
)
print(f"Created message, ID: {message.id}")

Created thread, ID: thread_T6wtYaDYXqg4p2lDniSog5eZ
Created message, ID: msg_vQdetwJ7D5V7AXVC6OtSmomk


In [60]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

Run finished with status: RunStatus.COMPLETED
Deleted agent
Messages: {'object': 'list', 'data': [{'id': 'msg_9CAE9idtk2zKfQcELLjrkTVe', 'object': 'thread.message', 'created_at': 1741353948, 'assistant_id': 'asst_BJ2u2TabJ3sXP5SRoai7DNUD', 'thread_id': 'thread_T6wtYaDYXqg4p2lDniSog5eZ', 'run_id': 'run_RVHglUKMIEktNGHUBwVMHvE2', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': "It seems that I'm unable to retrieve the weather information for Brussels, Belgium at the moment. However, the current date and time is something I can provide. \n\nWould you like me to send an email with just the current datetime, or do you have any other specific requests?", 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_vQdetwJ7D5V7AXVC6OtSmomk', 'object': 'thread.message', 'created_at': 1741353944, 'assistant_id': None, 'thread_id': 'thread_T6wtYaDYXqg4p2lDniSog5eZ', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Hello, send an email w

In [61]:
# Get the actual LLM completion message:
llm_completion_message = messages.get_last_text_message_by_role("assistant")
print(llm_completion_message["text"]["value"])

It seems that I'm unable to retrieve the weather information for Brussels, Belgium at the moment. However, the current date and time is something I can provide. 

Would you like me to send an email with just the current datetime, or do you have any other specific requests?


<h3>Code interpreter</h3>

<h3>OpenAI defined tools</h3>

<h3>Azure Functions</h3>

<h2>Using enterprise features</h2>

<h3>Using your own resources</h3>

<h3>Tracing with Application Insights</h3>

<h3>Content filtering </h3>

<h3>Using virtual networks</h3>